# 1). Set Up the Environment

In [24]:
#import base libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [45]:
# import train dataset
google_review_data = pd.read_json("review-Vermont_10.json", lines=True)
advertisements_data = pd.read_json("advertisements.json")
irrelevant_data = pd.read_json("irrelevant.json")
rant_data = pd.read_json("rant_without_visit.json")

# 2). Exploratory Data Analysis

2a). Visualise the Google Review dataset...

In [46]:
print("\n----------FIRST FIVE INSTANCES IN GOOGLE REVIEW----------\n")
display(google_review_data.head())
print("\n----------GOOGLE REVIEW DATA INFORMATION----------\n")
display(google_review_data.info())
print("\n----------GOOGLE REVIEW DATA SUMMARY STATS----------\n")
display(google_review_data.describe())


----------FIRST FIVE INSTANCES IN GOOGLE REVIEW----------



,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.180269e+20,rebecca kerns,1620085852324,5,Always done right from wood stove to screens!,None,"{'time': 1620087641504, 'text': 'Good Evening,...",0x89e02445cb9db457:0x37f42bff4edf7a43
1,1.015327e+20,Peter DeForge,1580309946474,5,A great company to work with. Their sales and...,None,"{'time': 1580320228721, 'text': 'Good Afternoo...",0x89e02445cb9db457:0x37f42bff4edf7a43
2,1.154041e+20,Chad Goulette,1605195974445,5,Great place to do business with staff was grea...,None,"{'time': 1605195166792, 'text': 'Hi Chad! Tha...",0x89e02445cb9db457:0x37f42bff4edf7a43
3,1.047893e+20,Mark LaFountain,1593005848256,5,"Awesome Customer service, quick response, and ...",None,"{'time': 1593376422014, 'text': 'Mark, thank y...",0x89e02445cb9db457:0x37f42bff4edf7a43
4,1.089807e+20,Jeff R,1582059996120,5,"If you need a top quality job, by a group of p...",None,"{'time': 1582063833737, 'text': 'Good Afternoo...",0x89e02445cb9db457:0x37f42bff4edf7a43



----------GOOGLE REVIEW DATA INFORMATION----------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324725 entries, 0 to 324724
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  324725 non-null  float64
 1   name     324725 non-null  object 
 2   time     324725 non-null  int64  
 3   rating   324725 non-null  int64  
 4   text     178168 non-null  object 
 5   pics     9651 non-null    object 
 6   resp     26838 non-null   object 
 7   gmap_id  324725 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 19.8+ MB


None


----------GOOGLE REVIEW DATA SUMMARY STATS----------



,user_id,time,rating
count,3.247250e+05,3.247250e+05,324725.000000
mean,1.091867e+20,1.548171e+12,4.314346
std,5.313166e+18,3.945905e+10,0.996339
min,1.000018e+20,1.182960e+12,1.000000
25%,1.045707e+20,1.523215e+12,4.000000
50%,1.091955e+20,1.550804e+12,5.000000
75%,1.137058e+20,1.573925e+12,5.000000
max,1.184442e+20,1.631106e+12,5.000000



Key Takeaways

- Columns with missing data in the train dataset: text, pics and resp
- Train dataset contains 324725 examples with 8 variables

2b). Visualise the advertisements, irrelevant, and rant_without_visit datasets...

In [47]:
print("\n----------FIRST FIVE INSTANCES IN ADVERTISEMENTS----------\n")
display(advertisements_data.head())
print("\n----------ADVERTISEMENTS DATA INFORMATION----------\n")
display(advertisements_data.info())
print("\n----------ADVERTISEMENTS DATA SUMMARY STATS----------\n")
display(advertisements_data.describe())

print("\n----------FIRST FIVE INSTANCES IN IRRELEVANT----------\n")
display(irrelevant_data.head())
print("\n----------IRRELEVANT DATA INFORMATION----------\n")
display(irrelevant_data.info())
print("\n----------IRRELEVANT DATA SUMMARY STATS----------\n")
display(irrelevant_data.describe())

print("\n----------FIRST FIVE INSTANCES IN RANT----------\n")
display(rant_data.head())
print("\n----------RANT DATA INFORMATION----------\n")
display(rant_data.info())
print("\n----------RANT DATA SUMMARY STATS----------\n")
display(rant_data.describe())


----------FIRST FIVE INSTANCES IN ADVERTISEMENTS----------



,reviews
0,{'text': 'The food at Luigi's Pizzeria was dec...
1,{'text': 'I was at Oak Creek Dental for a clea...
2,{'text': 'The handyman at City Hardware helped...
3,{'text': 'Stopped by The Daily Grind Cafe for ...
4,{'text': 'The new tires I got from Pete's Auto...



----------ADVERTISEMENTS DATA INFORMATION----------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   reviews  101 non-null    object
dtypes: object(1)
memory usage: 940.0+ bytes


None


----------ADVERTISEMENTS DATA SUMMARY STATS----------



...

,reviews
0,{'text': 'Absolutely the worst place ever. I w...
1,{'text': 'I heard from a friend of a friend ab...
2,{'text': 'After reading some things online abo...
3,{'text': 'The service is notoriously awful. Ev...
4,{'text': 'I drove by this place and just from ...



----------RANT DATA INFORMATION----------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   reviews  100 non-null    object
dtypes: object(1)
memory usage: 932.0+ bytes


None


----------RANT DATA SUMMARY STATS----------



,reviews
count,100
unique,100
top,{'text': 'Absolutely the worst place ever. I w...
freq,1



Key Takeaways

- Advertisements, Irrelevant and Rant datasets contain 101, 100, and 100 examples respectively.

# 4). Preprocessing and Modeling 

In [ ]:
import ollama
import re
from sklearn.metrics import precision_score, recall_score, f1_score

4a). Preprocessing

In [ ]:
genuine_texts = [item for item in google_review_data[:200]["text"] if item is not None]
# According to deepseek r1, these reviews are all genuine

ad_reviews_list = advertisements_data["reviews"]
# Extract the 'text' from each dictionary in the list
ad_texts = [review["text"] for review in ad_reviews_list]

irre_reviews_list = irrelevant_data["reviews"]
# Extract the 'text' from each dictionary in the list
irre_texts = [review["text"] for review in irre_reviews_list]

rant_reviews_list = rant_data["reviews"]
# Extract the 'text' from each dictionary in the list
rant_texts = [review["text"] for review in rant_reviews_list]

# Construct our evaluation dataset
columns=['text', 'ad_score', 'irre_score', 'rant_score']
data = []
for text in genuine_texts[:100]:
    data.append([text, 0, 0, 0])
for text in ad_texts[:100]:
    data.append([text, 1, 0, 0])
for text in irre_texts[:100]:
    data.append([text, 0, 1, 0])
for text in rant_texts[:100]:
    data.append([text, 0, 0, 1])
evaluation_df = pd.DataFrame(data, columns=columns).sample(frac=1.0, random_state=42)
print(evaluation_df)
X, y = evaluation_df["text"], evaluation_df.drop(columns=["text"])

# Visualise Evaluation Set
print("\n----------FIRST FIVE INSTANCES IN EVALUATION SET----------\n")
display(evaluation_df.head())
print("\n----------EVALUATION SET DATA INFORMATION----------\n")
display(evaluation_df.info())
print("\n----------EVALUATION SET DATA SUMMARY STATS----------\n")
display(evaluation_df.describe())

                                                  text  ad_score  irre_score  \
209  The price of gasoline is absolutely outrageous...         0           1   
280  My car is making a weird rattling noise. I hop...         0           1   
33                        Beautiful piece of property.         0           0   
210  I finally got my pumpkin seeds to sprout in my...         0           1   
93   Brent is very friendly and always willing to h...         0           0   
..                                                 ...       ...         ...   
71   It's quiet, not many people.  The path could u...         0           0   
106  The massage at Serenity Spa was relaxing, but ...         1           0   
270  My neighbor's lawn is looking exceptionally gr...         0           1   
348  I would rather have nothing than have to deal ...         0           0   
102  The handyman at City Hardware helped me find s...         1           0   

     rant_score  
209           0  
280

,text,ad_score,irre_score,rant_score
209,The price of gasoline is absolutely outrageous...,0,1,0
280,My car is making a weird rattling noise. I hop...,0,1,0
33,Beautiful piece of property.,0,0,0
210,I finally got my pumpkin seeds to sprout in my...,0,1,0
93,Brent is very friendly and always willing to h...,0,0,0



----------EVALUATION SET DATA INFORMATION----------

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 209 to 102
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        400 non-null    object
 1   ad_score    400 non-null    int64 
 2   irre_score  400 non-null    int64 
 3   rant_score  400 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 15.6+ KB


None


----------EVALUATION SET DATA SUMMARY STATS----------



,ad_score,irre_score,rant_score
count,400.000000,400.000000,400.000000
mean,0.250000,0.250000,0.250000
std,0.433555,0.433555,0.433555
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.250000,0.250000,0.250000
max,1.000000,1.000000,1.000000


4b). Evaluating Model: `deepseek-r1:14b`

In [ ]:
# Prompting and Prediction
batch_size = 10

i = 0
pred_data = []
columns = ['ad_score', 'irre_score', 'rant_score']
y_pred = pd.DataFrame(pred_data, columns=columns)

# Start with the initial prompt
content = """
You are an expert review classifier. Your task is to analyze user reviews and assign a confidence level for three specific categories: "advertisement", "irrelevant", and "rant without visit".

**Categories & Examples:**
-   **advertisement**: "Best pizza! Visit www.pizzapromo.com for discounts!"
-   **irrelevant**: "I love my new phone, but this place is too noisy."
-   **rant without visit**: "Never been here, but i heard it's terrible."

**Input:**
A list of reviews.

**Task:**
For the provided reviews, analyze the text, user, and business response fields to determine a confidence level (Low, or High) for each category.

**Output Instructions:**
- **DO NOT** include any other text, explanations, or commentary before or after the JSON.
- **DO NOT** include a `reviews` or any other key for the main JSON array.
- The output must be a single JSON array `[]` containing one JSON object `{}` for each review. All reviews must be analysed.
- Each object must have exactly three keys: `"advertisement"`, `"irrelevant"`, and `"rant without visit"`.
- Each value must be one of the strings: `"Low"` or `"High"`.
- The format must be identical to the example output.

**Example Input:**
"Well they did a bang up job, can't complain! Thank you, Mid-State for your hospitality and your service!", "Awesome place to get a car!!"

**Example Output:**
```json
[
  {
    "advertisement": "Low",
    "irrelevant": "Low",
    "rant without visit": "Low"
  },
  {
    "advertisement": "Low",
    "irrelevant": "Low",
    "rant without visit": "Low"
  }
]
"""

"""
account for this kind of output:
1. **"I think reality TV is contributing to the decline of modern society. Bring back scripted shows."** / **Review 1**
   - **Advertisement:** Low  
   - **Irrelevant:** High  
   - **Rant without Visit:** Low  
"""
messages=[
    {'role': 'user', 'content': content}
]

# 2. Make the first call
response1 = ollama.chat(model='deepseek-r1:14b', messages=messages) # requires the model to be installed locally in ollama
#print(response1['message']['content'])

# 3. Append the model's response to the message history
messages.append({'role': 'assistant', 'content': response1['message']['content']})

while i < len(evaluation_df): #len(evaluation_df)
  print("Processing reviews " + str(i+1) + " to " + str(i+batch_size))
  # 4. Add the next user prompt to test reviews
  content2 = ", ".join([f'"{item}"' for item in X[i:i+batch_size]])
  messages.append({'role': 'user', 'content': content2})

  # 5. Make the second call, passing the entire updated message list
  response2 = ollama.chat(model='deepseek-r1:14b', messages=messages)
  messages.pop() # remove the previous message

  index = response2['message']['content'].find("</think>") + len("</think>")
  actual_output = response2['message']['content'][index:]
  print(actual_output)

  raw_pred = []
  words_to_find = ["advertisement", "irrelevant", "rant without visit"]
  for word in words_to_find:
    pattern = r'(\"|\*\*|)' + re.escape(word) + r'(\"|\*\*|):'
    for match in re.finditer(pattern, actual_output, re.IGNORECASE):
        #print(f"Found '{match.group()}' at index {match.start()}")
        #print(actual_output[match.end():match.end()+10].lower())
        if "lo" in actual_output[match.end():match.end()+10].lower():
          raw_pred.append(0)
        if "hi" in actual_output[match.end():match.end()+10].lower():
          raw_pred.append(1)

  if len(raw_pred) != 3*batch_size:
    print(f"length incorrect, expected {3*batch_size} but got {len(raw_pred)}")
    continue

  columns = ['ad_score', 'irre_score', 'rant_score']
  for j in range(batch_size):
    y_pred.loc[len(y_pred)] = [raw_pred[j], raw_pred[j+batch_size], raw_pred[j+2*batch_size]]

  i += batch_size

Processing reviews 1 to 10
Processing reviews 11 to 20


In [ ]:
# Results

y_truth = y[:50]

for category in y_truth.columns:
    print(f"--- Metrics for {category} ---")

    true_labels = y_truth[category]
    pred_labels = y_pred[category]

    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels)

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}\n")



    ad_score  irre_score  rant_score
0          1           0           0
1          0           0           0
2          0           0           0
3          0           0           0
4          0           1           0
5          0           1           0
6          1           1           0
7          0           0           0
8          0           0           0
9          0           0           1
10         1           0           0
11         0           0           0
12         0           0           1
13         1           0           0
14         0           0           0
15         0           0           1
16         1           0           0
17         0           0           1
18         1           0           0
19         1           0           0
20         0           1           0
21         0           0           0
22         0           0           0
23         0           1           0
24         1           0           0
25         1           0           0
2